In [1]:
import pickle, gzip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os
import re, ast
import itertools
import seaborn as sns
from matplotlib import rcParams
from qiskit.visualization import plot_histogram
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from seaborn import set_theme
from matplotlib import style

rcParams.update({'figure.autolayout': True})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
#rc('text', usetex=True)
rcParams['text.usetex'] = True

In [17]:
def parse_text_file_real_machine_good(filename):
    fp = open(filename,'r')
    results = dict()
    for line in fp:
        line=line.strip()
        
        #circuit:BV7-0 qubit:6 theta:0.0 phi:0.0 second_qubit:1 second_theta:0.0 second_phi:0.0 counts:{'000000': 4, '010000': 1, '000010': 11, '111010': 11, '101000': 31, '001010': 26, '101010': 887, '100010': 38, '100011': 1, '101011': 9, '100000': 2, '101110': 3} 
        m = re.search('circuit:(.*) qubit:(.*) theta:(.*) phi:(.*) second_qubit:(.*) second_theta:(.*) second_phi:(.*) counts:({.*})', line)
        if m:
            circuit_name = 'BV7'#m.group(1)
            qubit = int(m.group(2))
            theta =  float(m.group(3))
            phi =  float(m.group(4))
            second_qubit = int(m.group(5))
            second_theta =  float(m.group(6))
            second_phi =  float(m.group(7))
            output = ast.literal_eval(m.group(8))
            dict_key = circuit_name
            if dict_key not in results.keys():
                # output_gold
                results[dict_key] = {'name':circuit_name, 'gate_type':'u', 'output_injections_noise':[], 'injection_qubit':[], 'theta': [], 'phi':[], 'second_injection_qubit':[], 'second_theta':[], 'second_phi':[]}
            if (theta == 0) and (phi == 0) and (second_phi == 0) and (second_theta == 0):                
                results[dict_key]['output_gold_noise'] = output
            results[dict_key]['output_injections_noise'].append(output)
            results[dict_key]['injection_qubit'].append( qubit )
            results[dict_key]['theta'].append( theta )
            results[dict_key]['phi'].append( phi )
            results[dict_key]['second_injection_qubit'].append( second_qubit )
            results[dict_key]['second_theta'].append( second_theta )
            results[dict_key]['second_phi'].append( second_phi )
            
    return results

def QVF_michelson_contrast(gold_bitstring, answer, shots):    
    # Sort the answer, position 0 has the highest bitstring, position 1 the second highest
    answer_sorted = sorted(answer, key=answer.get, reverse=True)
    
    # If gold bitstring is not in answer, percentage is zero
    if gold_bitstring not in answer:
        good_percent = 0
    else:
        good_percent = answer[gold_bitstring]/shots
        
    if answer_sorted[0] == gold_bitstring: # gold bitstring has the highest count (max)
        # next bitstring is the second highest
        next_percent = answer[answer_sorted[1]]/shots 
        next_bitstring = answer_sorted[1]
    else: # gold bitstring has NOT the highest count (not max)
        next_percent = answer[answer_sorted[0]]/shots 
        next_bitstring = answer_sorted[0]
    qvf = (good_percent - next_percent) / (good_percent + next_percent)    
    return 1 - (qvf+1)/2, next_bitstring

####### original version
# # Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast_single_injection(df, circuit_name, phi, theta):
    dfFilter = df[(df.circuit_name==circuit_name) & (df.first_phi==phi) & (df.first_theta==theta)]
    #print('computing QVF for',circuit_name)
    #print(dfFilter)
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()
    #print(QVF['QVF_circuit'],sum(dfFilter['success']),len(dfFilter['success']))
    #print('-----------')
    
    qubits = set(dfFilter['first_qubit_injected'])    
    for q in qubits:
        QVF['QVF_qubit_'+str(q)] = dfFilter[dfFilter.first_qubit_injected==q]['QVF'].mean()
    #QVF['gold_ok'] = dfFilter['gold_success'][0]
    return QVF

#######multi-injection version
# Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast_double_injection(df, circuit_name, phi_0, theta_0, phi_1, theta_1):
    dfFilter = df[(df.circuit_name == circuit_name) & (df.first_phi == phi_0) & (df.first_theta == theta_0)& (df.second_phi == phi_1) & (df.second_theta == theta_1)]
    #print('computing QVF for',circuit_name)
    #print(dfFilter)
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()
    #print(QVF['QVF_circuit'],sum(dfFilter['success']),len(dfFilter['success']))
    #print('-----------')

    qubits = set(dfFilter['first_qubit_injected'])
    for q in qubits:
        QVF['QVF_qubit_' +
            str(q)] = dfFilter[dfFilter.first_qubit_injected == q]['QVF'].mean()
    #QVF['gold_ok'] = dfFilter['gold_success'][0]
    return QVF

# Read pickled data and store results in a dataframe
def build_DF_newQVFRealMachine(data):
    results = []
    shots = 1024
    gold_bitstring = max(data['output_gold_noise'], key=data['output_gold_noise'].get)#check
    original_gold_percentage = data['output_gold_noise'][gold_bitstring]/shots

    for i, answer in enumerate(data['output_injections_noise']):
        qvf, next_bitstring = QVF_michelson_contrast(gold_bitstring, answer, shots)
        max_key = max(answer, key=answer.get)
        output_percentage = answer[max_key]/shots
        next_bitstring_percentage = answer[next_bitstring]/shots
        if gold_bitstring not in answer:
            gold_percentage = 0
        else:
            gold_percentage = answer[gold_bitstring]/shots
        result = {'gold_bitstring':gold_bitstring
                , 'gold_count_percentage':gold_percentage
                , 'original_gold_count_percentage':original_gold_percentage
                , 'next_bitstring': next_bitstring
                , 'next_bitstring_percentage': next_bitstring_percentage
                , 'QVF':qvf
                , 'first_qubit_injected':data['injection_qubit'][i]
                #, 'gate_injected':data['gate_type']
                , 'first_phi':data['phi'][i]
                , 'first_theta':data['theta'][i]
                , 'second_qubit_injected':data['second_injection_qubit'][i]
                , 'second_phi':data['second_phi'][i]
                , 'second_theta':data['second_theta'][i]
                #, 'lambda':0
                , 'circuit_name':data['name']
                }
        results.append(result)
    return pd.DataFrame(results)

                #, 'first_qubit_injected':data['circuits_injections'][i].metadata['first_qubit']                
                #, 'first_phi':data['circuits_injections'][i].metadata['first_phi']
                #, 'first_theta':data['circuits_injections'][i].metadata['first_theta']
                #, 'second_qubit_injected':data['circuits_injections'][i].metadata['second_qubit']                
                #, 'second_phi':data['circuits_injections'][i].metadata['second_phi']
                #, 'second_theta':data['circuits_injections'][i].metadata['second_theta']


# Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast(df, circuit_name, phi, theta):
    dfFilter = df[(df.circuit_name==circuit_name) & (df.phi==phi) & (df.theta==theta)]
    #print('computing QVF for',circuit_name)
    #print(dfFilter)
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()
    #print(QVF['QVF_circuit'],sum(dfFilter['success']),len(dfFilter['success']))
    #print('-----------')
    
    qubits = set(dfFilter['qubit_injected'])    
    for q in qubits:
        QVF['QVF_qubit_'+str(q)] = dfFilter[dfFilter.qubit_injected==q]['QVF'].mean()
    #QVF['gold_ok'] = dfFilter['gold_success'][0]
    return QVF

In [15]:
files_to_process = [
    '../results/double_fi_BV_7qubits.txt', 
]
# read all data and insert it into one dataframe
df_newQVF = pd.DataFrame()
for filename in files_to_process:
    data = parse_text_file_real_machine_good(filename)
    #print('data:',data)
    for d in data:
        df_newQVF = pd.concat([df_newQVF, build_DF_newQVFRealMachine(data[d])], ignore_index=True)
df_newQVF        

,gold_bitstring,gold_count_percentage,original_gold_count_percentage,next_bitstring,next_bitstring_percentage,QVF,first_qubit_injected,first_phi,first_theta,second_qubit_injected,second_phi,second_theta,circuit_name
0,101010,0.866211,0.875,100010,0.037109,0.041081,6,0.000000,0.000000,1,0.000000,0.000000,BV7
1,101010,0.868164,0.875,101000,0.043945,0.048180,6,0.000000,0.000000,3,0.000000,0.000000,BV7
2,101010,0.861328,0.875,001010,0.056641,0.061702,6,0.000000,0.000000,5,0.000000,0.000000,BV7
3,101010,0.843750,0.875,100010,0.064453,0.070968,6,0.000000,0.000000,1,0.000000,0.000000,BV7
4,101010,0.855469,0.875,100010,0.035156,0.039474,6,0.000000,0.000000,3,0.000000,0.000000,BV7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347797,101010,0.000977,0.875,011010,0.849609,0.998852,5,3.141593,3.141593,4,2.094395,3.141593,BV7
347798,101010,0.002930,0.875,011010,0.849609,0.996564,5,3.141593,3.141593,4,2.356194,3.141593,BV7
347799,101010,0.001953,0.875,011010,0.823242,0.997633,5,3.141593,3.141593,4,2.617994,3.141593,BV7
347800,101010,0.000000,0.875,011010,0.850586,1.000000,5,3.141593,3.141593,4,2.879793,3.141593,BV7


In [18]:
# Get available circuits and parameters used for injection (phi and theta)
phi_list = list(set(df_newQVF.first_phi))
phi_list.sort(reverse=True)
theta_list = list(set(df_newQVF.first_theta))
theta_list.sort()
circuits = list(set(df_newQVF.circuit_name))
circuits.sort()

results = []
for circuit in circuits:
    for phi in phi_list:
        for theta in theta_list:
            qvf = compute_QVF_michelson_contrast_single_injection(df_newQVF, circuit, phi, theta)
            qvf['circuit_name'] = circuit
            qvf['first_phi'] = phi
            qvf['first_theta'] = theta
            #qvf['threshold'] = threshold
            results.append(qvf)

new_qvfDF_noise = pd.DataFrame(results)
new_qvfDF_noise = new_qvfDF_noise[sorted(list(new_qvfDF_noise.columns))]
new_qvfDF_noise
#compute_QVF_michelson_contrast

,QVF_circuit,QVF_qubit_0,QVF_qubit_1,QVF_qubit_2,QVF_qubit_3,QVF_qubit_4,QVF_qubit_5,QVF_qubit_6,circuit_name,first_phi,first_theta
0,0.771571,0.809767,0.674544,0.812586,0.675779,0.718525,0.675292,0.864632,BV7,3.141593,0.000000
1,0.763648,0.803892,0.670219,0.806193,0.669267,0.711909,0.669489,0.853360,BV7,3.141593,0.261799
2,0.742533,0.776628,0.662530,0.779557,0.660712,0.694204,0.658014,0.823479,BV7,3.141593,0.523599
3,0.711971,0.734832,0.652146,0.737413,0.652912,0.674505,0.644643,0.775497,BV7,3.141593,0.785398
4,0.669405,0.682496,0.637245,0.683591,0.635238,0.642612,0.622466,0.712065,BV7,3.141593,1.047198
...,...,...,...,...,...,...,...,...,...,...,...
164,0.729520,0.756951,0.767432,0.758161,0.764273,0.752650,0.761997,0.676904,BV7,0.000000,2.094395
165,0.823793,0.851809,0.861387,0.860569,0.865985,0.857980,0.861198,0.759404,BV7,0.000000,2.356194
166,0.888758,0.929215,0.930009,0.928518,0.933729,0.925470,0.928130,0.817693,BV7,0.000000,2.617994
167,0.925467,0.963912,0.973337,0.968174,0.972173,0.964868,0.967948,0.849689,BV7,0.000000,2.879793


In [19]:
# comparing distribution of single FI vs double FI
dirHistPlots = '../plots/double_FI/histograms/'
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots) 


#new_qvfDF_noise_single
#new_qvfDF_noise
circs = [new_qvfDF_noise]
qvf_tmp = list()
for i, dfCirc in enumerate(circs):
    qvf_tmp.append(dfCirc)
    qvf_tmp[i] = qvf_tmp[i].pivot('first_phi', 'first_theta', 'QVF_circuit')
    qvf_tmp[i].columns.name = '$\\theta$ shift'
    qvf_tmp[i].index.name = '$\\phi$ shift'

    all_values = []
    for column in qvf_tmp[i]:
        this_column_values = qvf_tmp[i][column].tolist()
        all_values += this_column_values
    one_column_df = pd.DataFrame(all_values)
    print('mean:',one_column_df.mean()[0],' std:',one_column_df.std()[0])
print()

fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sns.set(font_scale=1.3)
#ax = 
colorsDist = ['black', 'red', 'green', 'blue']
for i, data in enumerate(qvf_tmp):
    sns.distplot(data, bins=256, color=colorsDist[i])
#sns.distplot(qvf_tmp2, bins=256, color='red')
plt.xlim(0, 1)

tmpFileName = dirHistPlots+'BV7qubits_distribution_histogram'+'.pdf'
fig.savefig(tmpFileName, bbox_inches = 'tight')
plt.close()
#print(tmpFileName)

mean: 0.5426496633161131  std: 0.20322955212423233



/home/dagoliveira/.local/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
